# ConversationBufferMemory

이 메모리는 메시지를 저장한 다음 변수에 메시지를 추출할 수 있게 해줍니다.

먼저 문자열로 추출할 수 있습니다.


In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

history.add_user_message("안녕?")
history.add_ai_message("반가워!")

print(history.messages)

#Memory 전체 구조가 ChatMessageHistory 객체 기반으로 완전히 바뀜
#모든 기능을 Runnable- LCEL중심으로 만들고, 메모리는 사용자 정의로 단순하게

[HumanMessage(content='안녕?', additional_kwargs={}, response_metadata={}), AIMessage(content='반가워!', additional_kwargs={}, response_metadata={})]


In [3]:
import sys
sys.executable

import langchain
import anthropic
from dotenv import load_dotenv


In [4]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1) 메모리 생성
history = InMemoryChatMessageHistory()

# 2) human 메시지 저장
history.add_user_message(
    "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)

# 3) ai 메시지 저장
history.add_ai_message(
    "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
)

# 4) 출력 확인
history.messages


[HumanMessage(content='안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={})]

In [5]:
history.messages 
# = memory

print([m.content for m in history.messages])
# 기존 방식은 print(memory.load_memory_variables({})["history"])



['안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', '안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?']


memory 의 `load_memory_variables({})` 함수는 메시지 히스토리를 반환합니다.


In [6]:
# 'history' 키에 저장된 대화 기록을 확인합니다.
# print(memory.load_memory_variables({})["history"]) # 더 이상 존재하지 않는다
print(history.messages, "\n")

#참고: 대화 내용을 옛날 방식처럼 텍스트 형식으로 출력하려면 다음과 같이 할 수 있습니다.
conversation_text = "\n".join([f"{m.type}: {m.content}" for m in history.messages])
print(conversation_text)


[HumanMessage(content='안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={})] 

human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
ai: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?


`save_context(inputs, outputs)` 메서드를 사용하여 대화 기록을 저장할 수 있습니다.

- 이 메서드는 `inputs`와 `outputs` 두 개의 인자를 받습니다.
- `inputs`는 사용자의 입력을, `outputs`는 AI의 출력을 저장합니다.
- 이 메서드를 사용하면 대화 기록이 `history` 키에 저장됩니다.
- 이후 `load_memory_variables` 메서드를 사용하여 저장된 대화 기록을 확인할 수 있습니다.


In [7]:
# inputs: dictionary(key: "human" or "ai", value: 질문)
# outputs: dictionary(key: "ai" or "human", value: 답변)
# memory.save_context(
#     inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
#     outputs={
#         "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
#     },
# ) 
# save_context 메소드도 사라짐

history.add_user_message("네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?")
history.add_ai_message(
    "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
)




In [8]:
# 2개의 대화를 저장합니다.
# memory.save_context(
#     inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
#     outputs={
#         "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
#     outputs={
#         "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
#     },
# )

# history.add_user_message("사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?")
# history.add_ai_message(
#     "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
# )

# save_context 대체 코드
history.add_user_message("사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?")
history.add_ai_message(
    "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
)

# save_context 대체 코드
history.add_user_message("인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?")
history.add_ai_message(
    "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
)

In [9]:
# history에 저장된 대화 기록을 확인합니다.
#print(memory.load_memory_variables({})["history"])



conversation_text = "\n".join([f"{m.type}: {m.content}" for m in history.messages])
print(conversation_text)

human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
ai: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
human: 네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?
ai: 감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.
human: 사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?
ai: 업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
human: 인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?
ai: 본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다.


In [18]:
# 추가로 2개의 대화를 저장합니다.
# memory.save_context(
#     inputs={"human": "정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"},
#     outputs={
#         "ai": "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
#     },
# )
# memory.save_context(
#     inputs={"human": "모든 절차를 완료했습니다. 계좌가 개설된 건가요?"},
#     outputs={
#         "ai": "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
#     },
# )
# 4번 대화
history.add_user_message("정보를 모두 입력했습니다. 다음 단계는 무엇인가요?")
history.add_ai_message(
    "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
)

# 5번 대화
history.add_user_message("모든 절차를 완료했습니다. 계좌가 개설된 건가요?")
history.add_ai_message(
    "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
)




In [10]:
# history에 저장된 대화 기록을 확인합니다.
#print(memory.load_memory_variables({})["history"])

conversation_text = "\n".join([f"{m.type}: {m.content}" for m in history.messages])
print(conversation_text)

human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
ai: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
human: 네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?
ai: 감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.
human: 사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?
ai: 업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
human: 인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?
ai: 본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다.


`return_messages=True` 로 설정하면 `HumanMessage` 와 `AIMessage` 객체를 반환합니다.


In [11]:
# ConversationBufferMemory → InMemoryChatMessageHistory 대체
# return_messages=True → 기본적으로 messages 리스트 리턴함 = 원래는 구분되어 있었지만 최신 형태는 True가 디폴트 값

# True 메세지 객체 리스트 / 위의 False는 문자열 형태

# memory = ConversationBufferMemory(return_messages=True)

# memory.save_context(
#     inputs={
#         "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
#     },
#     outputs={
#         "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
#     },
# )

# memory.save_context(
#     inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
#     outputs={
#         "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
#     },
# )

# memory.save_context(
#     inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
#     outputs={
#         "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
#     },
# )





In [12]:
# history에 저장된 대화 기록을 확인합니다.
#print(memory.load_memory_variables({})["history"])
conversation_text = "\n".join([f"{m.type}: {m.content}" for m in history.messages])
print(conversation_text)

human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
ai: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
human: 네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?
ai: 감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.
human: 사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?
ai: 업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
human: 인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?
ai: 본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다.


In [13]:
from langchain_core.prompts import ChatPromptTemplate

ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 봇입니다"),
        ("human", "대한민국의 수도는 어디야?"),
    ]
)

c:\Users\jeffr\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-SGRq4Tjy-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 AI 봇입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='대한민국의 수도는 어디야?'), additional_kwargs={})])

## Chain 에 적용


In [14]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

`ConversationChain`을 사용하여 대화를 진행합니다.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter # 딕셔너리에서 특정 키의 값을 추출하기 위해 사용

# Runnable은 dictionary 형태의 입력과 출력을 요구하는데 OpenAI의 Chat 모델은 메시지 리스트를 입력으로 받습니다.
# 따라서 Runnable 체인 앞뒤에 메시지 리스트를 추출하고 삽입하는 로직이 필요하다.

#이전 방식: RunnableWithMessageHistory를 체인의 중간에 넣으려고 시도하거나, 메시지 추출과 분리하려고 했습니다.

# 최종 방식 (래핑): RunnableWithMessageHistory를 완성된 뼈대 체인 (base_chain) 전체를 감싸는 Wrapper처럼 사용합니다.

# LLM과 메시지 변환 체인을 먼저 정의
# 1. 입력 딕셔너리에서 메시지 리스트를 추출하고 
# 2. LLM에 전달하는 기본적인 체인
base_chain = itemgetter("messages") | ChatOpenAI(temperature=0, model="gpt-4o-mini")

# 2. 이 기본 체인을 RunnableWithMessageHistory로 래핑합니다.
#    RunnableWithMessageHistory는 base_chain 앞뒤에 히스토리 로직을 추가합니다.
conversation = RunnableWithMessageHistory(
    base_chain,
    # 세션 관리를 위한 람다 함수 (InMemoryChatMessageHistory 사용)
    lambda session_id: InMemoryChatMessageHistory(), 
    
    # 딕셔너리에서 메시지를 가져올 키 (입력과 출력 모두)
    input_messages_key="messages",
)





session_id_key = "user1"

print(
    conversation.invoke( # 메모리, 세션, 런타임 설정이 prompt와 완전히 분리되었기 때문
        # 여기서 LLM이 아닌 RunnableWithMessageHistory에 전체 딕셔너리를 전달합니다.
        {"messages": [HumanMessage(content="안녕하세요, 계좌 개설하고 싶어요.")]},
        config={"configurable": {"session_id": session_id_key}}
    ).content
)


print(
    conversation.invoke(
        {"messages": [HumanMessage(content="계속 설명해 주세요.")]},
        config={"configurable": {"session_id": session_id_key}}
    ).content
)

안녕하세요! 계좌 개설에 대해 도와드리겠습니다. 어떤 종류의 계좌를 원하시는지, 예를 들어 개인 계좌, 기업 계좌, 저축 계좌 등 구체적으로 말씀해 주시면 더 정확한 정보를 제공해 드릴 수 있습니다. 또한, 어떤 은행을 고려하고 계신지도 알려주시면 좋습니다.
물론입니다! 어떤 주제에 대해 설명을 원하시는지 말씀해 주시면, 그에 맞춰 자세히 설명해 드리겠습니다. 예를 들어, 과학, 기술, 역사, 문화 등 다양한 분야에 대해 이야기할 수 있습니다. 원하는 주제를 알려주세요!


In [17]:
# # 대화를 시작합니다.
# response = conversation.predict(
#     input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
# )
# print(response)

# 대화를 시작합니다.
# predict() 대신 invoke()를 사용하고, 메시지 리스트를 담은 딕셔너리 형태로 입력을 전달
from langchain_core.messages import HumanMessage # HumanMessage 클래스를 임포트

session_id_key = "user1" # 사용할 세션 ID를 지정

response = conversation.invoke(
    {
        "messages": [
            HumanMessage(content="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?")
        ]
    },
    config={"configurable": {"session_id": session_id_key}}
)

# Runnable.invoke()의 결과는 BaseMessage 객체이므로, 내용(content)을 추출하여 출력합니다.
print(response.content)

안녕하세요! 비대면으로 은행 계좌를 개설하는 방법은 다음과 같습니다:

1. **은행 선택**: 먼저, 비대면 계좌 개설을 지원하는 은행을 선택하세요. 대부분의 주요 은행들은 모바일 앱이나 웹사이트를 통해 비대면 계좌 개설 서비스를 제공합니다.

2. **필요 서류 준비**: 일반적으로 신분증(주민등록증, 운전면허증 등)과 같은 신분 확인 서류가 필요합니다. 일부 은행에서는 추가적인 서류를 요구할 수 있으니, 해당 은행의 요구 사항을 확인하세요.

3. **온라인 신청**: 선택한 은행의 웹사이트나 모바일 앱에 접속하여 계좌 개설 신청을 시작하세요. 필요한 정보를 입력하고, 서류를 업로드합니다.

4. **신원 확인**: 은행에서 신원 확인 절차를 진행합니다. 이 과정에서 비대면으로 본인 인증을 위한 영상 통화나 인증 절차가 있을 수 있습니다.

5. **계좌 개설 완료**: 모든 절차가 완료되면, 계좌 개설이 완료되었다는 안내를 받을 것입니다. 이후 계좌 번호와 관련 정보를 확인할 수 있습니다.

6. **모바일 뱅킹 설정**: 계좌 개설 후, 모바일 뱅킹 앱을 다운로드하고 로그인하여 필요한 설정을 완료하세요.

각 은행마다 절차와 요구 사항이 다를 수 있으니, 선택한 은행의 공식 웹사이트나 고객센터를 통해 구체적인 정보를 확인하는 것이 좋습니다. 도움이 필요하시면 언제든지 질문해 주세요!


이전의 대화 기록을 기억하고 있는지 확인합니다.


In [ ]:
# # 이전 대화내용을 불렛포인트로 정리해 달라는 요청을 보냅니다.
# response = conversation.predict(
#     input="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요."
# )
# print(response)

# 이전 대화내용을 불렛포인트로 정리해 달라는 요청을 보냅니다.
from langchain_core.messages import HumanMessage # HumanMessage 클래스를 임포트합니다.

# 사용할 세션 ID는 이전 대화와 동일하게 "user1"을 사용합니다.
session_id_key = "user1" 

# 대화를 시작합니다.
# predict() 대신 invoke()를 사용하고, 메시지 리스트를 담은 딕셔너리 형태로 입력을 전달합니다.
response = conversation.invoke(
    {
        "messages": [
            HumanMessage(content="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요.")
        ]
    },
    config={"configurable": {"session_id": session_id_key}}
)

# Runnable.invoke()의 결과는 BaseMessage 객체이므로, 내용(content)을 추출하여 출력합니다.
print(response.content)

# response = conversation.invoke(
#     {
#         "messages": [
#             HumanMessage(content="제가 처음에 질문했던 **비대면 계좌 개설 절차**에 대한 내용을 불렛포인트 형식으로 정리하여 알려주세요.")
#         ]
#     },
#     config={"configurable": {"session_id": session_id_key}}
# )

# # 답변 내용 출력
# print(response.content)

물론입니다! 이전 답변의 내용을 불렛포인트 형식으로 정리해 드리겠습니다. 하지만 어떤 내용을 정리해야 하는지 알려주시면 더 정확하게 도와드릴 수 있습니다. 특정 주제나 내용을 말씀해 주시면 그에 맞춰 정리해 드리겠습니다.
비대면 계좌 개설 절차는 다음과 같이 정리할 수 있습니다:

- **은행 선택**: 비대면 계좌 개설이 가능한 은행을 선택합니다.
- **모바일 앱 다운로드**: 해당 은행의 모바일 앱을 다운로드합니다.
- **회원 가입**: 앱에서 회원 가입 절차를 진행합니다.
- **신원 인증**:
  - 본인 확인을 위한 신분증(주민등록증, 운전면허증 등) 촬영 및 업로드
  - 얼굴 인식 또는 비디오 인증 절차 진행
- **계좌 유형 선택**: 원하는 계좌 유형(예: 입출금 통장, 적금 등)을 선택합니다.
- **개설 정보 입력**: 개인 정보(이름, 주소, 연락처 등) 및 계좌 관련 정보 입력
- **약관 동의**: 서비스 이용 약관 및 개인정보 처리 방침에 동의합니다.
- **계좌 개설 완료**: 모든 절차가 완료되면 계좌 개설이 완료되며, 계좌 번호가 발급됩니다.
- **계좌 사용 시작**: 발급된 계좌 번호로 입출금 및 기타 금융 거래를 시작할 수 있습니다.

이 절차는 은행에 따라 다소 차이가 있을 수 있으니, 각 은행의 안내를 참고하는 것이 좋습니다.
